# 03 — Costo, Latencia y Alucinacion

**Objetivo**: Cuantificar las tres metricas criticas de cualquier sistema LLM: cuanto cuesta, cuanto tarda, y cuanto alucina.

## Contenido
1. Token economics de gpt-5-mini
2. Clase `CostTracker`
3. Distribucion de latencia (p50/p95/p99)
4. Alucinacion sin/con contexto (RAG basico)
5. LLM-as-judge para hallucination scoring
6. Dashboard con matplotlib

In [ ]:
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()
MODEL = "gpt-5-mini"

print("=" * 60)
print("METRICAS: COSTO, LATENCIA, ALUCINACION")
print("=" * 60)

## 1. Token Economics

Modelo `gpt-5-mini`:
- Input: $0.15 / 1M tokens
- Output: $0.60 / 1M tokens
- Cached input: $0.075 / 1M tokens

Referencia para comparar:
- `gpt-5`: $2.00 / $8.00 por 1M tokens (13x mas caro)
- `gpt-4o-mini`: $0.15 / $0.60 por 1M tokens (referencia anterior)

In [ ]:
# ============================================================
# CostTracker: rastreo de costos por request
# ============================================================

class CostTracker:
    """Rastrea costos acumulados de llamadas a OpenAI."""

    # Precios por millon de tokens
    PRICING = {
        "gpt-5-mini": {"input": 0.15, "output": 0.60},
        "gpt-5": {"input": 2.00, "output": 8.00},
    }

    def __init__(self, model: str = "gpt-5-mini"):
        self.model = model
        self.calls: list[dict] = []

    def track(self, usage, latency_ms: float, label: str = "") -> dict:
        """Registra una llamada y calcula costo."""
        prices = self.PRICING.get(self.model, self.PRICING["gpt-5-mini"])
        costo_input = usage.prompt_tokens * prices["input"] / 1_000_000
        costo_output = usage.completion_tokens * prices["output"] / 1_000_000

        entry = {
            "label": label,
            "input_tokens": usage.prompt_tokens,
            "output_tokens": usage.completion_tokens,
            "total_tokens": usage.total_tokens,
            "costo_input": costo_input,
            "costo_output": costo_output,
            "costo_total": costo_input + costo_output,
            "latencia_ms": latency_ms,
        }
        self.calls.append(entry)
        return entry

    @property
    def total_costo(self) -> float:
        return sum(c["costo_total"] for c in self.calls)

    @property
    def total_tokens(self) -> int:
        return sum(c["total_tokens"] for c in self.calls)

    def resumen(self) -> str:
        latencias = [c["latencia_ms"] for c in self.calls]
        return (
            f"Llamadas: {len(self.calls)} | "
            f"Tokens: {self.total_tokens:,} | "
            f"Costo: ${self.total_costo:.6f} | "
            f"Latencia media: {np.mean(latencias):.0f}ms"
        )


tracker = CostTracker(MODEL)
print("CostTracker inicializado")
print(f"Precios {MODEL}: {CostTracker.PRICING[MODEL]}")

## 2. Distribucion de Latencia

Ejecutamos N llamadas identicas para medir la variabilidad de latencia.

In [ ]:
# ============================================================
# MEDICION DE LATENCIA (10 llamadas)
# ============================================================

preguntas = [
    "Que es un transformer?",
    "Explica backpropagation en una oracion.",
    "Que es attention en deep learning?",
    "Define overfitting.",
    "Que es gradient descent?",
    "Que es un embedding?",
    "Que es fine-tuning?",
    "Que es tokenizacion?",
    "Que es un prompt?",
    "Que es inference en ML?",
]

print("Ejecutando 10 llamadas para medir latencia...\n")

for i, pregunta in enumerate(preguntas):
    t0 = time.time()
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Responde en 1-2 oraciones en español."},
            {"role": "user", "content": pregunta},
        ],
        max_tokens=100,
    )
    latencia = (time.time() - t0) * 1000
    entry = tracker.track(response.usage, latencia, label=f"q{i+1}")
    print(f"  [{i+1:2d}] {latencia:6.0f}ms | {entry['total_tokens']:4d} tokens | ${entry['costo_total']:.6f}")

print(f"\n{tracker.resumen()}")

In [ ]:
# ============================================================
# PERCENTILES DE LATENCIA
# ============================================================

latencias = [c["latencia_ms"] for c in tracker.calls]

print("=" * 60)
print("DISTRIBUCION DE LATENCIA")
print("=" * 60)
print(f"  p50 (mediana):  {np.percentile(latencias, 50):6.0f} ms")
print(f"  p90:            {np.percentile(latencias, 90):6.0f} ms")
print(f"  p95:            {np.percentile(latencias, 95):6.0f} ms")
print(f"  p99:            {np.percentile(latencias, 99):6.0f} ms")
print(f"  min:            {min(latencias):6.0f} ms")
print(f"  max:            {max(latencias):6.0f} ms")
print(f"  std:            {np.std(latencias):6.0f} ms")

## 3. Alucinacion: Sin Contexto vs Con Contexto

Comparamos las respuestas del LLM cuando:
- **Sin contexto**: responde solo con sus parametros (propenso a alucinar)
- **Con contexto**: recibe informacion de referencia (RAG basico)

In [ ]:
# ============================================================
# DEMO DE ALUCINACION
# ============================================================

contexto_real = """
LangGraph es una libreria de LangChain para construir aplicaciones con LLMs 
usando grafos dirigidos. Permite definir nodos (funciones), edges (transiciones), 
y estados tipados. Fue lanzada en 2024. Soporta ciclos, lo que la diferencia 
de DAGs tradicionales. Usa StateGraph como clase principal.
"""

pregunta_test = "Quien creo LangGraph y en que año? Que version actual tiene?"

# Sin contexto (propenso a alucinar)
t0 = time.time()
r_sin = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "Responde en español con datos especificos."},
        {"role": "user", "content": pregunta_test},
    ],
)
lat_sin = (time.time() - t0) * 1000
tracker.track(r_sin.usage, lat_sin, label="sin_contexto")

# Con contexto (RAG basico)
t0 = time.time()
r_con = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": f"Responde SOLO con la informacion del contexto. Si no esta en el contexto, di 'No tengo esa informacion.'\n\nContexto:\n{contexto_real}"},
        {"role": "user", "content": pregunta_test},
    ],
)
lat_con = (time.time() - t0) * 1000
tracker.track(r_con.usage, lat_con, label="con_contexto")

print("=" * 60)
print("COMPARACION: SIN vs CON CONTEXTO")
print("=" * 60)
print(f"\nPregunta: {pregunta_test}")
print(f"\n--- SIN CONTEXTO (puede alucinar) ---")
print(r_sin.choices[0].message.content)
print(f"\n--- CON CONTEXTO (RAG basico) ---")
print(r_con.choices[0].message.content)

## 4. LLM-as-Judge: Hallucination Scoring

Usamos un segundo LLM para evaluar si la respuesta original alucino.

In [ ]:
# ============================================================
# LLM-AS-JUDGE: evaluacion de alucinacion
# ============================================================

def evaluar_alucinacion(pregunta: str, respuesta: str, contexto: str | None = None) -> dict:
    """
    Usa un LLM como juez para evaluar alucinacion.

    Returns:
        Dict con score (1-5), justificacion, y metricas.
    """
    if contexto:
        prompt_juez = f"""Evalua si la siguiente respuesta contiene alucinaciones 
(informacion inventada o no respaldada por el contexto).

Contexto de referencia:
{contexto}

Pregunta: {pregunta}
Respuesta: {respuesta}

Evalua con un score de 1 a 5:
1 = Completamente alucinada (todo inventado)
2 = Mayormente alucinada
3 = Parcialmente correcta, parcialmente alucinada
4 = Mayormente correcta
5 = Sin alucinacion (todo respaldado por el contexto)

Responde en JSON: {{"score": N, "justificacion": "..."}}"""
    else:
        prompt_juez = f"""Evalua la plausibilidad de la siguiente respuesta.

Pregunta: {pregunta}
Respuesta: {respuesta}

Score 1-5 (1=probablemente inventado, 5=probablemente correcto).
Responde en JSON: {{"score": N, "justificacion": "..."}}"""

    t0 = time.time()
    r = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Eres un evaluador critico. Responde solo en JSON valido."},
            {"role": "user", "content": prompt_juez},
        ],
        response_format={"type": "json_object"},
    )
    lat = (time.time() - t0) * 1000
    tracker.track(r.usage, lat, label="judge")

    try:
        resultado = json.loads(r.choices[0].message.content)
    except json.JSONDecodeError:
        resultado = {"score": 0, "justificacion": "Error parsing JSON"}

    return resultado


# Evaluar ambas respuestas
print("=" * 60)
print("EVALUACION LLM-AS-JUDGE")
print("=" * 60)

eval_sin = evaluar_alucinacion(pregunta_test, r_sin.choices[0].message.content, contexto_real)
print(f"\nSIN contexto → Score: {eval_sin.get('score', '?')}/5")
print(f"  Justificacion: {eval_sin.get('justificacion', 'N/A')}")

eval_con = evaluar_alucinacion(pregunta_test, r_con.choices[0].message.content, contexto_real)
print(f"\nCON contexto → Score: {eval_con.get('score', '?')}/5")
print(f"  Justificacion: {eval_con.get('justificacion', 'N/A')}")

## 5. Dashboard de Metricas

In [ ]:
# ============================================================
# DASHBOARD: 3 paneles
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Distribucion de latencia
latencias_q = [c["latencia_ms"] for c in tracker.calls if c["label"].startswith("q")]
axes[0].hist(latencias_q, bins=8, color="#2196F3", edgecolor="white", alpha=0.8)
axes[0].axvline(np.percentile(latencias_q, 50), color="red", linestyle="--", label=f"p50={np.percentile(latencias_q, 50):.0f}ms")
axes[0].axvline(np.percentile(latencias_q, 95), color="orange", linestyle="--", label=f"p95={np.percentile(latencias_q, 95):.0f}ms")
axes[0].set_xlabel("Latencia (ms)")
axes[0].set_ylabel("Frecuencia")
axes[0].set_title("Distribucion de Latencia")
axes[0].legend()

# Panel 2: Costo por llamada
costos = [c["costo_total"] * 1000 for c in tracker.calls if c["label"].startswith("q")]
axes[1].bar(range(1, len(costos) + 1), costos, color="#4CAF50", alpha=0.8)
axes[1].set_xlabel("Llamada #")
axes[1].set_ylabel("Costo (milésimas USD)")
axes[1].set_title("Costo por Llamada")
axes[1].axhline(np.mean(costos), color="red", linestyle="--", label=f"Media={np.mean(costos):.3f}")
axes[1].legend()

# Panel 3: Hallucination scores
scores = [eval_sin.get("score", 0), eval_con.get("score", 0)]
bars = axes[2].bar(["Sin contexto", "Con contexto (RAG)"], scores, color=["#FF5722", "#4CAF50"], alpha=0.8)
axes[2].set_ylabel("Hallucination Score (1-5)")
axes[2].set_title("Alucinacion: Sin vs Con Contexto")
axes[2].set_ylim(0, 5.5)
for bar, score in zip(bars, scores):
    axes[2].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.1,
                 f"{score}/5", ha="center", fontweight="bold")

plt.tight_layout()
plt.savefig("../data/dashboard_metricas.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"\nResumen final: {tracker.resumen()}")

## Takeaways

1. **Costo**: gpt-5-mini es extremadamente barato (~$0.0001 por llamada simple). Pero en agentes multi-paso, el costo se multiplica.
2. **Latencia**: La variabilidad es alta (p95 puede ser 3-5x el p50). Disenar para el peor caso, no el promedio.
3. **Alucinacion**: Sin contexto, el LLM inventa datos especificos (versiones, fechas). Con RAG, se limita a lo que sabe.
4. **LLM-as-Judge**: Es una herramienta poderosa para evaluacion automatica, pero agrega costo y latencia adicional.
5. **Dashboard**: Visualizar metricas es esencial para tomar decisiones informadas sobre arquitectura.